# Imports

In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
import numpy as np
import functools
from difflib import SequenceMatcher

In [2]:
load_dotenv()

True

In [3]:
DATABASE_PASSWORD=os.getenv("DATABASE_PASSWORD")
MYSQL_USER=os.getenv("MYSQL_USER")
DATABASE_NAME=os.getenv("DATABASE_NAME")

# Utils

In [4]:
def check_connection(database_name=None):
    """Décorateur qui vérifie si la connexion MySQL est active et peut changer de base de données

    Args:
        database_name (str, optional): Nom de la base de données à utiliser.
    """
    def decorator(fonction):
        @functools.wraps(fonction)
        def wrapper(*args, **kwargs):
            # Récupération de l'objet connexion
            if args:  
                connection = args[0]
            elif "connection" in kwargs:  
                connection = kwargs["connection"]
            else:
                raise ValueError("L'argument 'connection' est requis")

            # Vérifier si la connexion est active
            if not connection.is_connected():
                print("🔄 Connexion fermée. Tentative de reconnexion...")
                try:
                    connection.reconnect()
                    print("✅ Connexion rétablie.")
                except mysql.connector.Error as err:
                    raise ConnectionError(f"Impossible de se reconnecter : {err}")

            # Changer la base de données si nécessaire
            if database_name and connection.database != database_name:
                try:
                    print(f"🔄 Changement de base de données vers {database_name}")
                    connection.database = database_name
                except mysql.connector.Error as err:
                    raise ConnectionError(f"Impossible de changer de base de données : {err}")

            return fonction(*args, **kwargs)

        return wrapper

    return decorator

In [5]:
def get_closest_name(target_name, names):
    """Retourne le nom le plus proche de target_name dans la liste names (via distance de Levenshtein normalisée)

    Args:
        target_name (str): Le nom à comparer
        names (list): La liste des noms à comparer
    
    Returns:
        str: Le nom le plus proche de target_name
    """
    ratios = np.array([SequenceMatcher(None, name, target_name).ratio() for name in names])
    match = names[np.argmax(ratios)]

    return match

# Données en CSV

In [6]:
df = pd.read_csv("personnes.csv")

df

,personnes,caracteres
0,John Hammond,"vieux, milliardaire"
1,Ellie Satler,"jeune, scientifique"
2,Alan Grant,"vieux, scientifique"
3,Ian Malcolm,"jeune, scientifique"


# Création base de données et tables

In [7]:
def create_connection(**kwargs):
    """Crée une connexion à la base de données MySQL"""
    try:
        connection = mysql.connector.connect(
            host=kwargs['host'],
            user=kwargs['user'],
            password=kwargs['password'],
            database=kwargs['database']
        )
        return connection
    except Error as e:
        print(f"Erreur lors de la connexion à MySQL: {e}")
        return None

In [8]:
params_conn = {"host": "localhost", "user": MYSQL_USER, "password": DATABASE_PASSWORD, "database": ""}

In [9]:
conn = create_connection(**params_conn)

In [10]:
conn.is_connected()

True

In [11]:
@check_connection()
def create_database(connection, db_name):
    """Crée une base de données

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        db_name (str): Le nom de la base de données

    Returns:
        None
    """
    
    cursor = connection.cursor(buffered=True)
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        print(f"Base de données {db_name} créée avec succès")
        connection.database = db_name
    except Error as e:
        print(f"Erreur lors de la création de la base de données: {e}")
    finally:
        cursor.close()

In [12]:
@check_connection(database_name=DATABASE_NAME)
def execute_query(connection, query):
    """Exécute une requête SQL Simple

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        query (str): La requête SQL à exécuter

    Returns:
        None
    
    """
    
    cursor = connection.cursor(buffered=True)
    try:
        cursor.execute(query)
        connection.commit()
        print("Requête exécutée avec succès")
    except Error as e:
        print(f"Erreur lors de l'exécution de la requête: {e}")
    finally:
        cursor.close()

In [13]:
create_database(conn, "personnes")

Base de données personnes créée avec succès


In [14]:
@check_connection(database_name=DATABASE_NAME)
def create_tables(connection):
    """Crée les tables nécessaires"""
    assert connection.is_connected() , "La connexion à la base de données est fermée"

    # Table personnes
    create_personnes_table = """
    CREATE TABLE IF NOT EXISTS personnes (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nom VARCHAR(100) NOT NULL UNIQUE
    );
    """
    
    # Table caractéristiques
    create_caracteristiques_table = """
    CREATE TABLE IF NOT EXISTS caracteristiques (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nom VARCHAR(100) NOT NULL UNIQUE
    );
    """
    
    # Table de jonction
    create_junction_table = """
    CREATE TABLE IF NOT EXISTS personnes_caracteristiques (
        personne_id INT,
        caracteristique_id INT,
        PRIMARY KEY (personne_id, caracteristique_id),
        FOREIGN KEY (personne_id) REFERENCES personnes(id),
        FOREIGN KEY (caracteristique_id) REFERENCES caracteristiques(id)
    );
    """
    
    execute_query(connection, create_personnes_table)
    execute_query(connection, create_caracteristiques_table)
    execute_query(connection, create_junction_table)

In [15]:
create_tables(conn)

Requête exécutée avec succès
Requête exécutée avec succès
Requête exécutée avec succès


In [16]:
conn.is_connected()

True

# Insertion des données

In [17]:
@check_connection(database_name=DATABASE_NAME)
def insert_person(connection, nom):
    """Insère une personne dans la table personnes

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        nom (str): Le nom de la personne

    Returns:
        L'id de la personne insérée

    """
    
    cursor = connection.cursor()
    try:
        cursor.execute("INSERT IGNORE INTO personnes (nom) VALUES (%s)", (nom,))
        connection.commit()
        return cursor.lastrowid
    except Error as e:
        print(f"Erreur lors de l'insertion de la personne: {e}")
        return None
    finally:
        cursor.close()

In [18]:
@check_connection(database_name=DATABASE_NAME)
def get_person_id(connection, nom):
    """Récupère l'ID d'une personne

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        nom (str): Le nom de la personne

    Returns:
        L'id de la personne
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute("SELECT id FROM personnes WHERE nom = %s", (nom,))
        result = cursor.fetchone()
        return result[0] if result else None
    except Error as e:
        print(f"Erreur lors de la récupération de l'ID de la personne: {e}")
        return None
    finally:
        cursor.close()

In [19]:
@check_connection(database_name=DATABASE_NAME)
def insert_caracteristique(connection, nom):
    """Insère une caractéristique dans la table caractéristiques

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        nom (str): Le nom de la caractéristique

    Returns:
        L'id de la caractéristique insérée
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute("INSERT IGNORE INTO caracteristiques (nom) VALUES (%s)", (nom,))
        connection.commit()
        return cursor.lastrowid
    except Error as e:
        print(f"Erreur lors de l'insertion de la caractéristique: {e}")
        return None
    finally:
        cursor.close()

In [20]:
@check_connection(database_name=DATABASE_NAME)
def get_caracteristique_id(connection, nom):
    """Récupère l'ID d'une caractéristique

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        nom (str): Le nom de la caractéristique

    Returns:
        L'id de la caractéristique
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute("SELECT id FROM caracteristiques WHERE nom = %s", (nom,))
        result = cursor.fetchone()
        return result[0] if result else None
    except Error as e:
        print(f"Erreur lors de la récupération de l'ID de la caractéristique: {e}")
        return None
    finally:
        cursor.close()

In [21]:
@check_connection(database_name=DATABASE_NAME)
def link_person_to_caracteristique(connection, personne_id, caracteristique_id):
    """Lie une personne à une caractéristique
    
    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        personne_id (int): L'id de la personne
        caracteristique_id (int): L'id de la caractéristique
    
    Returns:
        None
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute(
            "INSERT IGNORE INTO personnes_caracteristiques (personne_id, caracteristique_id) VALUES (%s, %s)",
            (personne_id, caracteristique_id)
        )
        connection.commit()
    except Error as e:
        print(f"Erreur lors de la liaison personne-caractéristique: {e}")
    finally:
        cursor.close()

In [22]:
conn.is_connected()

True

## Insertion effective

In [23]:
# Parcourir chaque ligne du DataFrame
for _, row in df.iterrows():
    personne = row['personnes']
    
    # Insérer la personne
    insert_person(conn, personne)
    personne_id = get_person_id(conn, personne)
    
    # Traiter les caractéristiques
    caracteristiques_list = row['caracteres'].split(',')
    for caracteristique in caracteristiques_list:
        caracteristique = caracteristique.strip().strip('"')
        
        # Insérer la caractéristique
        insert_caracteristique(conn, caracteristique)
        caracteristique_id = get_caracteristique_id(conn, caracteristique)
        
        # Lier la personne à la caractéristique
        if personne_id and caracteristique_id:
            link_person_to_caracteristique(conn, personne_id, caracteristique_id)

# Requêtes de test

In [24]:
params_conn = {"host": "localhost", "user": MYSQL_USER, "password": DATABASE_PASSWORD, "database": DATABASE_NAME}

In [25]:
conn = create_connection(**params_conn)

In [26]:
conn.is_connected()

True

In [27]:
conn.database

'personnes'

## Requête Read simple (avec Pandas)

In [28]:
@check_connection(database_name=DATABASE_NAME)
def read_sql_(connection, query):
    """Exécute une requête SELECT et retourne les résultats sous forme de DataFrame Pandas

    Args:
        connection (mysql.connector.connection.MySQLConnection): La connexion MySQL
        query (str): La requête SELECT

    Returns:
        pd.DataFrame: Les résultats de la requête
    """
    
    return pd.read_sql(query, connection)

In [29]:
conn.is_connected()

True

In [30]:
conn.close()

In [31]:
conn.is_connected()

False

In [32]:
df = read_sql_(conn, "SELECT * FROM personnes")

🔄 Connexion fermée. Tentative de reconnexion...
✅ Connexion rétablie.


/var/folders/fj/crd4vf5s1h74s1h2w_9m3x6r0000gn/T/ipykernel_47512/908817662.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, connection)


In [33]:
df

,id,nom
0,3,Alan Grant
1,2,Ellie Satler
2,4,Ian Malcolm
3,1,John Hammond


## Récupérer les personnes ayant une caractéristique - Read

In [34]:
@check_connection(database_name=DATABASE_NAME)
def trouver_personnes_par_caracteristique(connection, caracteristique):
    """
    Récupère toutes les personnes ayant une caractéristique spécifique
    
    Args:
        connection: La connexion à la base de données
        caracteristique: Le nom de la caractéristique recherchée
        
    Returns:
        Une liste des noms de personnes ayant cette caractéristique
    """
    cursor = connection.cursor()
    try:
        query = """
        SELECT personnes.nom
        FROM personnes
        JOIN personnes_caracteristiques ON personnes.id = personnes_caracteristiques.personne_id
        JOIN caracteristiques ON personnes_caracteristiques.caracteristique_id = caracteristiques.id
        WHERE caracteristiques.nom = %s
        """
        cursor.execute(query, (caracteristique,))
        resultats = cursor.fetchall()
        resultats = [resultat[0] for resultat in resultats]

        if len(resultats) == 0:
            query="SELECT nom FROM caracteristiques WHERE nom=%s"
            cursor.execute(query, (caracteristique,))
            resultat_caracteristiques = cursor.fetchall()
            if not resultat_caracteristiques:
                print("La caractéristique n'a pas été trouvée...")
                query="SELECT nom FROM caracteristiques"
                cursor.execute(query)
                toutes_caracteristiques = cursor.fetchall()

                caracteristique_la_plus_proche = get_closest_name(caracteristique, [caracteristique[0] for caracteristique in toutes_caracteristiques])
                print("La caractéristique la plus proche est : ", caracteristique_la_plus_proche)
                return False
            else:
                print(f"Aucune personne trouvée pour la caractéristique {caracteristique}")
                return []


        return resultats
    except Error as e:
        print(f"Erreur lors de la recherche des personnes par caractéristique: {e}")
        return []
    finally:
        cursor.close()

In [37]:
trouver_personnes_par_caracteristique(conn, "vieux")

['John Hammond', 'Alan Grant']

## Récupérer les caractéristiques d'une personne - Read

In [35]:
@check_connection(database_name=DATABASE_NAME)
def trouver_caracteristiques_par_personne(connection, nom_personne):
    """
    Récupère toutes les caractéristiques d'une personne spécifique
    
    Args:
        connection: La connexion à la base de données
        nom_personne:str Le nom de la personne recherchée
        
    Returns:
        Une liste des caractéristiques de cette personne
    """
    cursor = connection.cursor()
    try:
        query = """
        SELECT caracteristiques.nom 
        FROM caracteristiques
        JOIN personnes_caracteristiques ON caracteristiques.id = personnes_caracteristiques.caracteristique_id
        JOIN personnes ON personnes_caracteristiques.personne_id = personnes.id
        WHERE personnes.nom = %s
        """
        cursor.execute(query, (nom_personne,))
        resultats = cursor.fetchall()
        resultats = [resultat[0] for resultat in resultats]

        if len(resultats) == 0:
            query="SELECT nom FROM personnes WHERE nom=%s"
            cursor.execute(query, (nom_personne,))
            resultat_personne = cursor.fetchall()
            if not resultat_personne:
                print("La personne n'a pas été trouvée...")
                query="SELECT nom FROM personnes"
                cursor.execute(query)
                toutes_personnes = cursor.fetchall()

                personne_la_plus_proche = get_closest_name(nom_personne, [personne[0] for personne in toutes_personnes])
                print("La personne la plus proche est : ", personne_la_plus_proche)
                return False
            else:
                print(f"Aucune caractéristique trouvée pour {nom_personne}")
                return []
            
        return resultats
    except Error as e:
        print(f"Erreur lors de la recherche des caractéristiques par personne: {e}")
        return []
    finally:
        cursor.close()

In [36]:
trouver_caracteristiques_par_personne(conn, "Ian Malcol")

La personne n'a pas été trouvée...
La personne la plus proche est :  Ian Malcolm


False

## Rajouter une personne avec une caractéristique - Create

In [38]:
@check_connection(database_name=DATABASE_NAME)
def ajouter_personne_avec_caracteristiques(connection, nom_personne, liste_caracteristiques):
    """
    Ajoute une nouvelle personne et l'associe à des caractéristiques (nouvelles ou existantes)
    
    Args:
        connection: La connexion à la base de données
        nom_personne: Le nom de la nouvelle personne à ajouter
        liste_caracteristiques: Liste des caractéristiques à associer à la personne
        
    Returns:
        bool: True si l'opération a réussi, False sinon
    """
    try:
        # Sélectionner la base de données si ce n'est pas déjà fait
        if not connection.database:
            connection.database = "personnes"
        
        # Ajouter la nouvelle personne
        insert_person(connection, nom_personne)
        personne_id = get_person_id(connection, nom_personne)
        
        if not personne_id:
            print(f"Erreur lors de l'ajout de la personne '{nom_personne}'.")
            return False
        
        # Pour chaque caractéristique
        for caracteristique in liste_caracteristiques:
            # Vérifier si la caractéristique existe déjà
            caracteristique_id = get_caracteristique_id(connection, caracteristique)
            
            # Si elle n'existe pas, l'ajouter
            if not caracteristique_id:
                insert_caracteristique(connection, caracteristique)
                caracteristique_id = get_caracteristique_id(connection, caracteristique)
                if not caracteristique_id:
                    print(f"Erreur lors de l'ajout de la caractéristique '{caracteristique}'.")
                    continue
            
            # Créer le lien entre la personne et la caractéristique
            link_person_to_caracteristique(connection, personne_id, caracteristique_id)
        
        print(f"La personne '{nom_personne}' a été ajoutée avec ses caractéristiques avec succès!")
        return True
        
    except Error as e:
        print(f"Erreur lors de l'ajout de la personne avec caractéristiques: {e}")
        return False

In [39]:
conn.is_connected()

True

In [40]:
ajouter_personne_avec_caracteristiques(conn, "Donald Gennaro", ["vieux", "avocat"])

La personne 'Donald Gennaro' a été ajoutée avec ses caractéristiques avec succès!


True

## Modifier les caractéristiques d'une personne - Update

In [41]:
@check_connection(database_name=DATABASE_NAME)
def modifier_caracteristiques_personne(connection, nom_personne, ajouter_caracteristiques=None, supprimer_caracteristiques=None):
    """
    Modifie les caractéristiques d'une personne existante dans la base de données
    
    Args:
        connection: La connexion à la base de données
        nom_personne: Le nom de la personne à modifier
        ajouter_caracteristiques: Liste des caractéristiques à ajouter (None si aucune)
        supprimer_caracteristiques: Liste des caractéristiques à supprimer (None si aucune)
        
    Returns:
        bool: True si l'opération a réussi, False sinon
    """
    try:
        # Sélectionner la base de données si ce n'est pas déjà fait
        if not connection.database:
            connection.database = "personnes"
        
        # Vérifier si la personne existe
        personne_id = get_person_id(connection, nom_personne)
        if not personne_id:
            print(f"La personne '{nom_personne}' n'existe pas dans la base de données.")
            return False
        
        # Supprimer des caractéristiques si demandé avec le paramètre booléen
        if supprimer_caracteristiques:
            for caracteristique in supprimer_caracteristiques:
                caracteristique_id = get_caracteristique_id(connection, caracteristique)
                if caracteristique_id:
                    # Supprimer le lien entre la personne et cette caractéristique
                    cursor = connection.cursor()
                    try:
                        delete_query = """
                        DELETE FROM personnes_caracteristiques 
                        WHERE personne_id = %s AND caracteristique_id = %s
                        """
                        cursor.execute(delete_query, (personne_id, caracteristique_id))
                        connection.commit()
                        print(f"Caractéristique '{caracteristique}' supprimée pour '{nom_personne}'")
                    except Error as e:
                        print(f"Erreur lors de la suppression de la caractéristique: {e}")
                    finally:
                        cursor.close()
                else:
                    print(f"Caractéristique '{caracteristique}' non trouvée, rien à supprimer.")
        
        # Ajouter des caractéristiques si demandé avec le paramètre booléen
        if ajouter_caracteristiques:
            for caracteristique in ajouter_caracteristiques:
                # Vérifier si la caractéristique existe déjà
                caracteristique_id = get_caracteristique_id(connection, caracteristique)
                
                # Si elle n'existe pas, l'ajouter
                if not caracteristique_id:
                    insert_caracteristique(connection, caracteristique)
                    caracteristique_id = get_caracteristique_id(connection, caracteristique)
                    if not caracteristique_id:
                        print(f"Erreur lors de l'ajout de la caractéristique '{caracteristique}'.")
                        continue
                
                # Vérifier si le lien existe déjà
                cursor = connection.cursor()
                try:
                    check_query = """
                    SELECT 1 FROM personnes_caracteristiques 
                    WHERE personne_id = %s AND caracteristique_id = %s
                    """
                    cursor.execute(check_query, (personne_id, caracteristique_id))
                    if cursor.fetchone():
                        print(f"La caractéristique '{caracteristique}' est déjà associée à '{nom_personne}'")
                    else:
                        # Créer le lien entre la personne et la caractéristique
                        link_person_to_caracteristique(connection, personne_id, caracteristique_id)
                        print(f"Caractéristique '{caracteristique}' ajoutée pour '{nom_personne}'")
                except Error as e:
                    print(f"Erreur lors de la vérification du lien: {e}")
                finally:
                    cursor.close()
        
        print(f"Modification des caractéristiques pour '{nom_personne}' terminée.")
        return True
        
    except Error as e:
        print(f"Erreur lors de la modification des caractéristiques: {e}")
        return False

In [43]:
modifier_caracteristiques_personne(conn, "Donald Gennaro", ajouter_caracteristiques=["peureux"], supprimer_caracteristiques=["vieux"])

Caractéristique 'vieux' supprimée pour 'Donald Gennaro'
Caractéristique 'peureux' ajoutée pour 'Donald Gennaro'
Modification des caractéristiques pour 'Donald Gennaro' terminée.


True

In [42]:
conn.is_connected()

True